In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.style as style
style.use('fivethirtyeight')
import plotly.express as px
import seaborn as sns
sns.set_style('whitegrid')

import os
import warnings
warnings.filterwarnings('ignore')
from joblib import Parallel, delayed

from xgboost import XGBRegressor
from sklearn.model_selection import KFold
import xgboost as xgb
from sklearn.metrics import r2_score
import lightgbm as lgbm
from lightgbm import LGBMRegressor

In [6]:
pd.set_option('max_rows', 300)
pd.set_option('max_columns', 300)

import glob

In [7]:
data_dir = 'data/'

In [5]:
# Import data
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
book_train = pd.read_parquet('data/book_train.parquet')
book_test = pd.read_parquet('data/book_test.parquet')
trade_train = pd.read_parquet('data/trade_train.parquet')
trade_test = pd.read_parquet('data/trade_test.parquet')
submission = pd.read_csv('data/sample_submission.csv')

KeyboardInterrupt: 

In [ ]:
data['wap'] = (data['price'] * data['size']) / data['size']

In [ ]:
data['wap1'] = (data['price1'] * data['size1']) / data['size1']

In [ ]:
data['wap2'] = (data['price2'] * data['size2']) / data['size2']